
The objective is to assemble products starting from unassembled ones, by solving a scheduling problem based on a configurable assembly line. Unassembled products arrive from a **Source** and must be delivered to a **Sink** once assembly is complete. In our setup, we assume the assembly line is already configured—each hexagonal cell is pre-defined with a set of available operations.

### The operations that a cell can perform are as follows:

- **Transport**: Transfers products from the Source (entry point) to the cell's carousel, or from the carousel to the Sink (exit point).
- **Carousel**: Moves the product from the cell’s transport system to its internal warehouse.
- **Warehouse**: Acts as a storage area within the hexagon where assembly operations are carried out.
- **Staubli**: A robotic arm responsible for assembly tasks such as attaching a motor cover or vacuum gripper.
- **Binpick**: Picks required components from a bin, which are essential for specific Staubli operations.

### Summary

A product arrives from the Source and is initially handled by the cell’s transport system. The carousel transfers it to the warehouse, where the Staubli robot carries out the necessary assembly tasks. During this process, Binpick retrieves essential components needed for Staubli’s operations. Once assembly is complete, the carousel moves the item back to the transport system, which then delivers it to the Sink.

### Input

The data is constructed to simulate a shift of operations for building five products. We extended the original formulation by introducing a dedicated cell for each product to be delivered. Additionally, we reduced the number of warehouses to only one, as opposed to the original formulation, which had two warehouses associated with each cell. While an additional warehouse is not currently required, the original approach can be reverted if needed.

Specifically, for each item, four different Staubli operations are required:
- Add motor cover
- Screw M5
- Add feeder
- Screw M3

Some of these operations require picking certain items from the bin:
- Pick M5
- Pick M3

Other constraints include:
- The succession of specific actions, such as using the carousel immediately after the transport system.
- Time dependecy bewteen actions, such as using the binpick action must happen before the Staubli action associated with.
- Ensuring that the resources within each cell cannot be used simultaneously.



In [38]:
import json
from old.model_mf import MFModelILP
import time
import plotly.express as px
import pandas as pd


In [ ]:
with open("../data/model.json") as json_file:
    data = json.load(json_file)



In [ ]:
list_jobs = []
list_constr = []
for name_job in data['nodes']:
    res = data['nodes'][name_job]['properties']['Resource']['value']
    product_instance = data['nodes'][name_job]['properties']['Product_instance']['value']
    duration = data['nodes'][name_job]['properties'][f'{name_job}_duration']['value']
    list_jobs.append([name_job,res,product_instance,duration])

for name_constr in data['edges']:
    type_constr = data['edges'][name_constr]['__type__']
    list_constr.append([name_constr,type_constr])

In [41]:
df_jobs = pd.DataFrame(list_jobs, columns=['job name', 'resource', 'product','duration'])
df_jobs

,job name,resource,product,duration
0,1_16_0_0,Source_Source,1_16_0,10.0
1,1_16_0_1,Cell_0_Transport,1_16_0,10.0
2,1_16_0_2,Cell_0_Carrousel,1_16_0,8.0
3,1_16_0_3,Cell_0_Staubli,1_16_0,33.0
4,1_16_0_4,Cell_0_Binpick,1_16_0,320.0
...,...,...,...,...
100,warehouse_for_1_16_4_6,Cell_0_Warehouse_1,1_16_4,NaN
101,filler_between_warehouse_for_1_16_4_5_and_ware...,Cell_0_Warehouse_1,1_16_4,NaN
102,warehouse_for_1_16_4_8,Cell_0_Warehouse_1,1_16_4,NaN
103,filler_between_warehouse_for_1_16_4_6_and_ware...,Cell_0_Warehouse_1,1_16_4,NaN


In [42]:
df_edge = pd.DataFrame(list_constr, columns=['constraint name', 'constraint type', ])
df_edge

,constraint name,constraint type
0,1_16_0_0_is_right_before_1_16_0_1,IsRightBefore
1,1_16_0_1_is_right_before_1_16_0_2,IsRightBefore
2,filler_between_1_16_0_2_and_warehouse_for_1_16...,IsRightAfter
3,filler_between_1_16_0_2_and_warehouse_for_1_16...,IsRightBefore
4,warehouse_for_1_16_0_3_is_synced_1_16_0_3,IsSynced
...,...,...
944,filler_between_warehouse_for_1_16_4_5_and_ware...,HasNoOverlap
945,filler_between_warehouse_for_1_16_4_5_and_ware...,HasNoOverlap
946,warehouse_for_1_16_4_8_has_no_overlap_filler_b...,HasNoOverlap
947,warehouse_for_1_16_4_8_has_no_overlap_filler_b...,HasNoOverlap


In [62]:
model_ilp = MFModelILP(data)

for solver in ['ortools','exact','gurobi','choco']:
    start = time.time()
    if model_ilp.solve(solver=solver,time_limit=60):
        end = time.time()
        print(f'Solving for {solver} with ILP formulation: {end-start} seconds and {model_ilp.get_makespan()} of makespan')
    else:
        print(f'Timeout reached for {solver} with ILP formulation')

Solving for ortools with ILP formulation: 1.347165584564209 seconds and 2764 of makespan
Solving for exact with ILP formulation: 10.726134777069092 seconds and 2764 of makespan
Solving for gurobi with ILP formulation: 1.3801870346069336 seconds and 2764 of makespan
Timeout reached for choco with ILP formulation


In [43]:
model_ilp = MFModelILP(data)
start = time.time()
for solver in ['ortools']:
    if model_ilp.solve(solver=solver,time_limit=60):
        end = time.time()
        print(f'Solving for {solver} with ILP formulation: {end-start} seconds and {model_ilp.get_makespan()} of makespan')
    else:
        print(f'Timeout reached for {solver} with ILP formulation')

Solving for ortools with ILP formulation: 1.938889503479004 seconds and 2764 of makespan


### The operations that a cell can perform are as follows:

- **Transport**: Transfers products from the Source (entry point) to the cell's carousel, or from the carousel to the Sink (exit point).
- **Carousel**: Moves the product from the cell’s transport system to its internal warehouse.
- **Warehouse**: Acts as a storage area within the hexagon where assembly operations are carried out.
- **Staubli**: A robotic arm responsible for assembly tasks such as attaching a motor cover or vacuum gripper.
- **Binpick**: Picks required components from a bin, which are essential for specific Staubli operations.

In [44]:
jobs = model_ilp.get_jobs()
info = model_ilp.get_info()
start, duration = model_ilp.get_dv()
to_show =  []
dict_job_start = {}

for index in range(len(start)):
    dict_job_start[index] = start[index].value()

for job,job_info in jobs.items():
    subject = info[job_info['__id__']][0]
    start_job = start[subject].value()
    end_job = start[subject].value() + duration[subject].value()
    resource = info[job_info['__id__']][1]
    product = info[job_info['__id__']][2]
    job_name = info[job_info['__id__']][4]
    to_show.append(dict(Task = f'Task {subject}', Start= start_job,End = end_job, Resource = resource, Product = product, Name=job_name))

# Convert the tasks to a DataFrame for easier manipulation
df = pd.DataFrame(to_show)

# Calculate the duration of each task
df['Duration'] = df['End'] - df['Start']

# Use px.bar to create the Gantt chart with item-based colors
fig = px.bar(df, x='Duration', y='Resource', color='Product', orientation='h',
             hover_data=['Start', 'End', 'Task','Name'], base='Start', title="Job Shop Schedule")

# Update the layout to set the background to black and adjust text/grid colors for visibility
fig.update_layout(
    plot_bgcolor='black',  # Set background color of the plot to black
    paper_bgcolor='black',  # Set the overall figure background to black
    xaxis=dict(showgrid=True, gridcolor='lightgray', title="Time", color='white'),  # Set grid lines and label color to light for visibility
    yaxis=dict(showgrid=True, gridcolor='lightgray', title="Machine", color='white', categoryorder='category ascending'),  # Set grid and label color
    font=dict(color='white'),  # Set general font color to white for readability
    title_font=dict(color='white')  # Ensure the title font is also visible
)

fig.show()

### Extension with time for switching Staubli head

In [18]:
model_ilp.restore_feasiblity(dict_job_start,model_ilp.get_info())
for solver in ['ortools']:
    start = time.time()
    if model_ilp.solve(solver=solver,time_limit=60):
        end = time.time()
        print(f'Solving for {solver} with ILP formulation: {end-start} seconds and {model_ilp.get_makespan()} of makespan')
    else:
        print(f'Timeout reached for {solver} with ILP formulation')

Solving for ortools with ILP formulation: 0.99556565284729 seconds and 3344 of makespan


In [19]:
jobs = model_ilp.get_jobs()
info = model_ilp.get_info()
start, duration = model_ilp.get_dv()
to_show =  []
dict_job_start = {}

for index in range(len(start)):
    dict_job_start[index] = start[index].value()

for job,job_info in jobs.items():
    subject = info[job_info['__id__']][0]
    start_job = start[subject].value()
    end_job = start[subject].value() + duration[subject].value()
    resource = info[job_info['__id__']][1]
    product = info[job_info['__id__']][2]
    job_name = info[job_info['__id__']][4]
    op_type_name = info[job_info['__id__']][3]
    to_show.append(dict(Task = f'Task {subject}', Start= start_job,End = end_job, Resource = resource, Product = product, Name=job_name, Op_Type = op_type_name))

# Convert the tasks to a DataFrame for easier manipulation
df = pd.DataFrame(to_show)

# Calculate the duration of each task
df['Duration'] = df['End'] - df['Start']

# Use px.bar to create the Gantt chart with item-based colors
fig = px.bar(df, x='Duration', y='Resource', color='Product', orientation='h',
             hover_data=['Start', 'End', 'Task','Name','Op_Type'], base='Start', title="Job Shop Schedule")

# Update the layout to set the background to black and adjust text/grid colors for visibility
fig.update_layout(
    plot_bgcolor='black',  # Set background color of the plot to black
    paper_bgcolor='black',  # Set the overall figure background to black
    xaxis=dict(showgrid=True, gridcolor='lightgray', title="Time", color='white'),  # Set grid lines and label color to light for visibility
    yaxis=dict(showgrid=True, gridcolor='lightgray', title="Machine", color='white', categoryorder='category ascending'),  # Set grid and label color
    font=dict(color='white'),  # Set general font color to white for readability
    title_font=dict(color='white')  # Ensure the title font is also visible
)

fig.show()

### Maximum parallelization

In [ ]:
with open("../data/data_max_parallel_new.json") as json_file:
    data = json.load(json_file)

In [ ]:
model_ilp = MFModelILP(data, parallel=True)
for solver in ['ortools']:
    start = time.time()
    if model_ilp.solve(solver=solver,time_limit=60):
        end = time.time()
        print(f'Solving for {solver} with ILP formulation: {end-start} seconds and {model_ilp.get_makespan()} of makespan')
    else:
        print(f'Timeout reached for {solver} with ILP formulation')

In [33]:
jobs = model_ilp.get_jobs()
info = model_ilp.get_info()
start, duration, job_resource,_ = model_ilp.get_dv()
info_resource = model_ilp.get_info_resource()
to_show =  []
dict_job_start = {}

for index in range(len(start)):
    dict_job_start[index] = start[index].value()

for job,job_info in jobs.items():
    subject = info[job_info['__id__']][0]
    start_job = start[subject].value()
    end_job = start[subject].value() + duration[subject].value()
    ris_id = job_resource[subject].value()
    resource = next((k for k, v in info_resource.items() if v == ris_id), None)
    product = info[job_info['__id__']][2]
    job_name = info[job_info['__id__']][4]
    to_show.append(dict(Task = f'Task {subject}', Start= start_job,End = end_job, Resource = resource, Product = product, Name=job_name))

# Convert the tasks to a DataFrame for easier manipulation
df = pd.DataFrame(to_show)

# Calculate the duration of each task
df['Duration'] = df['End'] - df['Start']

# Use px.bar to create the Gantt chart with item-based colors
fig = px.bar(df, x='Duration', y='Resource', color='Product', orientation='h',
             hover_data=['Start', 'End', 'Task','Name'], base='Start', title="Job Shop Schedule")

# Update the layout to set the background to black and adjust text/grid colors for visibility
fig.update_layout(
    plot_bgcolor='black',  # Set background color of the plot to black
    paper_bgcolor='black',  # Set the overall figure background to black
    xaxis=dict(showgrid=True, gridcolor='lightgray', title="Time", color='white'),  # Set grid lines and label color to light for visibility
    yaxis=dict(showgrid=True, gridcolor='lightgray', title="Machine", color='white', categoryorder='category ascending'),  # Set grid and label color
    font=dict(color='white'),  # Set general font color to white for readability
    title_font=dict(color='white')  # Ensure the title font is also visible
)
fig.update_layout(height=750)
fig.show()

In [36]:
model_ilp.restore_feasiblity(dict_job_start,model_ilp.get_info())
for solver in ['ortools']:
    start = time.time()
    if model_ilp.solve(solver=solver,time_limit=60):
        end = time.time()
        print(f'Solving for {solver} with ILP formulation: {end-start} seconds and {model_ilp.get_makespan()} of makespan')
    else:
        print(f'Timeout reached for {solver} with ILP formulation')

Solving for ortools with ILP formulation: 6.280385971069336 seconds and 910 of makespan


In [37]:
jobs = model_ilp.get_jobs()
info = model_ilp.get_info()
start, duration, job_resource,_ = model_ilp.get_dv()
info_resource = model_ilp.get_info_resource()
to_show =  []
dict_job_start = {}

for index in range(len(start)):
    dict_job_start[index] = start[index].value()

for job,job_info in jobs.items():
    subject = info[job_info['__id__']][0]
    start_job = start[subject].value()
    end_job = start[subject].value() + duration[subject].value()
    ris_id = job_resource[subject].value()
    resource = next((k for k, v in info_resource.items() if v == ris_id), None)
    product = info[job_info['__id__']][2]
    job_name = info[job_info['__id__']][4]
    to_show.append(dict(Task = f'Task {subject}', Start= start_job,End = end_job, Resource = resource, Product = product, Name=job_name))

# Convert the tasks to a DataFrame for easier manipulation
df = pd.DataFrame(to_show)

# Calculate the duration of each task
df['Duration'] = df['End'] - df['Start']

# Use px.bar to create the Gantt chart with item-based colors
fig = px.bar(df, x='Duration', y='Resource', color='Product', orientation='h',
             hover_data=['Start', 'End', 'Task','Name'], base='Start', title="Job Shop Schedule")

# Update the layout to set the background to black and adjust text/grid colors for visibility
fig.update_layout(
    plot_bgcolor='black',  # Set background color of the plot to black
    paper_bgcolor='black',  # Set the overall figure background to black
    xaxis=dict(showgrid=True, gridcolor='lightgray', title="Time", color='white'),  # Set grid lines and label color to light for visibility
    yaxis=dict(showgrid=True, gridcolor='lightgray', title="Machine", color='white', categoryorder='category ascending'),  # Set grid and label color
    font=dict(color='white'),  # Set general font color to white for readability
    title_font=dict(color='white')  # Ensure the title font is also visible
)
fig.update_layout(height=750)
fig.show()

### Disruption scenarios

In [34]:
df['Availability'] = df['Resource'].apply(lambda x: 'Unavailable' if str(x).startswith('Cell_0') else 'Available')
fig = px.bar(df, x='Duration', y='Resource', color='Availability', orientation='h',
             hover_data=['Start', 'End', 'Task', 'Name', 'Product'], base='Start', title="Job Shop Schedule")
fig = px.bar(df, x='Duration', y='Resource', color='Availability', orientation='h',
             hover_data=['Start', 'End', 'Task', 'Name', 'Product'], base='Start', title="Job Shop Schedule")
color_map = {
    'Available': 'green',
    'Unavailable': 'red'
}

fig = px.bar(df, x='Duration', y='Resource', color='Availability', color_discrete_map=color_map,
             orientation='h', hover_data=['Start', 'End', 'Task', 'Name', 'Product'], base='Start',
             title="Job Shop Schedule")
fig.update_layout(title="Job Shop Schedule - Disruption scenario for broken cell")
fig.update_layout(height=750)

In [35]:
df.loc[df['Task'] == 'Task 63', 'Start'] = 500

# Recalculate the end time (using the same duration)
df.loc[df['Task'] == 'Task 63', 'End'] = df['Start'] + df['Duration']

# Add a highlight column to tag Task 63
df['Highlight'] = df['Task'].apply(lambda x: 'Highlighted' if x == 'Task 63' else 'Normal')

fig = px.bar(df, x='Duration', y='Resource', color='Highlight', orientation='h',
             hover_data=['Start', 'End', 'Task', 'Name', 'Product'], base='Start', title="Job Shop Schedule")

color_map = {
    'Normal': 'green',  # Regular task color
    'Highlighted': 'red'  # Highlighted task color
}

fig = px.bar(df, x='Duration', y='Resource', color='Highlight', color_discrete_map=color_map,
             orientation='h', hover_data=['Start', 'End', 'Task', 'Name', 'Product'], base='Start',
             title="Job Shop Schedule - Disruption scenario for delayed entrance")

fig.update_layout(
    plot_bgcolor='black',  # Set background color of the plot to black
    paper_bgcolor='black',  # Set the overall figure background to black
    xaxis=dict(showgrid=True, gridcolor='lightgray', title="Time", color='white'),
    yaxis=dict(showgrid=True, gridcolor='lightgray', title="Machine", color='white', categoryorder='category ascending'),
    font=dict(color='white'),
    title_font=dict(color='white')
)
fig.update_layout(height=750)
fig.show()